In [1]:
import os
import psycopg2
from psycopg2.extras import DictCursor
from datetime import datetime, timezone


In [2]:
def get_db_connection():
    return psycopg2.connect(
        host=os.getenv("POSTGRES_HOST", "localhost"),
        database=os.getenv("POSTGRES_DB", "assistant"),
        user=os.getenv("POSTGRES_USER", "your_username"),
        password=os.getenv("POSTGRES_PASSWORD", "your_password"),
    )

In [4]:
# create tables
conn = get_db_connection()
try:
    with conn.cursor() as cur:
        # cur.execute("DROP TABLE IF EXISTS feedback")
        # cur.execute("DROP TABLE IF EXISTS conversations")

        cur.execute("""
            CREATE TABLE if not exists conversations (
                id TEXT PRIMARY KEY,
                question TEXT NOT NULL,
                answer TEXT NOT NULL,
                model_used TEXT NOT NULL,
                response_time FLOAT NOT NULL,
                relevance TEXT NOT NULL,
                relevance_explanation TEXT NOT NULL,
                prompt_tokens INTEGER NOT NULL,
                completion_tokens INTEGER NOT NULL,
                total_tokens INTEGER NOT NULL,
                eval_prompt_tokens INTEGER NOT NULL,
                eval_completion_tokens INTEGER NOT NULL,
                eval_total_tokens INTEGER NOT NULL,
                openai_cost FLOAT NOT NULL,
                timestamp TIMESTAMP WITH TIME ZONE NOT NULL
            )
        """)
        cur.execute("""
            CREATE TABLE if not exists feedback (
                id SERIAL PRIMARY KEY,
                conversation_id TEXT REFERENCES conversations(id),
                feedback INTEGER NOT NULL,
                timestamp TIMESTAMP WITH TIME ZONE NOT NULL
            )
        """)
    conn.commit()
finally:
    conn.close()

In [ ]:
def save_conversation(conversation_id, question, answer_data, timestamp=None):
    if timestamp is None:
        timestamp = datetime.now()

    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(
                """
                INSERT INTO conversations 
                (id, question, answer, model_used, response_time, relevance, 
                relevance_explanation, prompt_tokens, completion_tokens, total_tokens, 
                eval_prompt_tokens, eval_completion_tokens, eval_total_tokens, openai_cost, timestamp)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (
                    conversation_id,
                    question,
                    answer_data["answer"],
                    answer_data["model_used"],
                    answer_data["response_time"],
                    answer_data["relevance"],
                    answer_data["relevance_explanation"],
                    answer_data["prompt_tokens"],
                    answer_data["completion_tokens"],
                    answer_data["total_tokens"],
                    answer_data["eval_prompt_tokens"],
                    answer_data["eval_completion_tokens"],
                    answer_data["eval_total_tokens"],
                    answer_data["openai_cost"],
                    timestamp
                ),
            )
        conn.commit()
    finally:
        conn.close()

In [36]:
from app.db_ops import save_feedback

# def save_feedback(conversation_id, feedback, timestamp=None):
#     if timestamp is None:
#         timestamp = datetime.now()
# 
#     conn = get_db_connection()
#     try:
#         with conn.cursor() as cur:
#             cur.execute(
#                 "INSERT INTO feedback (conversation_id, feedback, timestamp) VALUES (%s, %s, COALESCE(%s, CURRENT_TIMESTAMP))",
#                 (conversation_id, feedback, timestamp),
#             )
#         conn.commit()
#     finally:
#         conn.close()


save_feedback('101f8a41-f606-4556-9552-0f91b13b1f19', 0, )


<connection object at 0x7fdfc40dee00; dsn: 'user=your_username password=xxx dbname=assistant host=localhost', closed: 1>

# elasticsearch


In [13]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-distilbert-cos-v1")

from tqdm.auto import tqdm

# es_client = Elasticsearch('http://elasticsearch:9200') 
es_client = Elasticsearch('http://localhost:9200')


/home/mikel/.virtualenvs/llm-zoomcamp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
questions_path = "data/questions.json"
import json

# Open and load the JSON file
with open(questions_path, 'r') as file:
    questions = json.load(file)



In [7]:
import hashlib

documents = []


def generate_document_id(doc):
    combined = f"{doc['question']}-{doc['answer'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id


for question in questions['questions']:
    question['text_vector'] = model.encode(question['answer']).tolist()
    question['doc_id'] = generate_document_id(question)
    documents.append(question)


In [8]:
es_client.info()


ObjectApiResponse({'name': '601241a3dbc7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '51NRbnGSQAq2SNCUrsaqIQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
documents[0].keys()

dict_keys(['question', 'answer', 'text_vector', 'doc_id'])

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answer": {"type": "text"},
            "question": {"type": "text"},
            "doc_id": {"type": "text"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [11]:
index_name = "faq"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq'})

In [12]:
for doc in documents:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [13]:
search_term = "can I return?"
vector_search_term = model.encode(search_term)

query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 100,
}

res = es_client.search(index=index_name, knn=query, source=["answer", "question", 'doc_id'])
res["hits"]["hits"]

[{'_index': 'faq',
  '_id': 'A4wTSJIBzLr7WNaMcv69',
  '_score': 0.8741443,
  '_source': {'question': 'Can I return a product if I changed my mind?',
   'answer': 'Yes, you can return a product if you changed your mind. Please ensure the product is in its original condition and packaging, and refer to our return policy for instructions.',
   'doc_id': '9a9ab0e5'}},
 {'_index': 'faq',
  '_id': 'I4wTSJIBzLr7WNaMdP4x',
  '_score': 0.82811165,
  '_source': {'question': 'Can I return a product if it was damaged due to improper use?',
   'answer': 'Our return policy generally covers products that are defective or damaged upon arrival. Damage due to improper use may not be eligible for a return. Please contact our customer support team for assistance.',
   'doc_id': '2caeef0d'}},
 {'_index': 'faq',
  '_id': 'EYwTSJIBzLr7WNaMc_6o',
  '_score': 0.81617165,
  '_source': {'question': 'Can I return a product if I no longer have the original packaging?',
   'answer': 'While returning a product in it

# Evaluate retrieval 

In [19]:
import pandas as pd
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-distilbert-cos-v1")

from tqdm.auto import tqdm

# es_client = Elasticsearch('http://elasticsearch:9200') 
es_client = Elasticsearch('http://localhost:9200')

df_ground_truth = pd.read_csv('data/ground_truth.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
index_name = "faq"


/home/mikel/.virtualenvs/llm-zoomcamp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [32]:
def elastic_search_knn(search_term, index_name="faq"):
    vector_search_term = model.encode(search_term)
    query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 100,
    }
    es_results = es_client.search(index=index_name, knn=query, source=["answer", "question", 'doc_id'])
    return [hit["_source"] for hit in es_results["hits"]["hits"]]


def elastic_search_text(query, index_name="faq"):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "answer"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    return [hit["_source"] for hit in response["hits"]["hits"]]


def elastic_search_hybrid(search_term, index_name="faq"):
    text_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": search_term,
                        "fields": ["question^3", "answer"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    vector_search_term = model.encode(search_term)

    vector_query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 100,
    }

    # Perform the search with the constructed query
    es_results = es_client.search(index=index_name,
                                  query=text_query,
                                  knn=vector_query,
                                  source=["answer", "question", 'doc_id'],
                                  )

    # Extract and return the results
    result_docs = [hit["_source"] for hit in es_results["hits"]["hits"]]

    return result_docs

In [33]:
def hit_rate_one(original_id, search_results):
    return 1 if original_id in search_results else 0


def mrr_one(original_id, search_results):
    mrr = 0
    for position in range(len(search_results)):
        if search_results[position] == original_id:
            mrr += 1 / (position + 1)
    return mrr





In [36]:
hit_rate_results_text = []
hit_rate_results_vector = []
hit_rate_results_vector_combined = []
mrr_results_text = []
mrr_results_vector = []
mrr_results_vector_combined = []

for index, row in df_ground_truth.iterrows():
    document_id = row['doc_id']
    question = row['question']

    knn_results = elastic_search_knn(question, index_name=index_name)
    knn_results = [item['doc_id'] for item in knn_results]

    text_results = elastic_search_text(question)
    text_results = [item['doc_id'] for item in text_results]

    # 
    hit_rate_vector = hit_rate_one(document_id, knn_results)
    hit_rate_results_vector.append(hit_rate_vector)
    mrr_vector = mrr_one(document_id, knn_results)
    mrr_results_vector.append(mrr_vector)

    hit_rate_text = hit_rate_one(document_id, text_results)
    hit_rate_results_text.append(hit_rate_text)

    mrr_text = mrr_one(document_id, text_results)
    mrr_results_text.append(mrr_text)

    knn_combined_results = elastic_search_hybrid(question)
    # knn_combined_results = [item['doc_id'] for item in knn_combined_results]
    # hit_rate_vector_combined = hit_rate_one(document_id, knn_combined_results)
    # hit_rate_results_vector_combined.append(hit_rate_vector_combined)
    # mrr_vector_combined = mrr_one(document_id, knn_combined_results)
    # mrr_results_vector_combined.append(mrr_vector_combined)



BadRequestError: BadRequestError(400, 'parsing_exception', '[size] query malformed, no start_object after query name')

In [19]:
df_ground_truth['hit_rate_results_vector'] = hit_rate_results_vector
df_ground_truth['hit_rate_text'] = hit_rate_results_text

df_ground_truth['mrr_results_vector'] = mrr_results_vector
df_ground_truth['mrr_text'] = mrr_results_text



In [20]:
df_ground_truth.describe()

,hit_rate_results_vector,hit_rate_text,mrr_results_vector,mrr_text
count,400.000000,400.000000,400.000000,400.000000
mean,0.940000,0.847500,0.832292,0.709333
std,0.237784,0.359955,0.311742,0.393253
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000
